In [3]:
from flipside import Flipside
import pandas as pd
import os

In [8]:
API_KEY = os.environ.get('FLIPSIDE_API_KEY')

Pull data from Flipside. We first get balances of all uCVX, pxCVX, naked lpxcvx and Curve lpxcvx LP token holders at the last block eligible to vote on the Prisma whitelist proposal.

In [56]:
flipside = Flipside(API_KEY, "https://api-v2.flipsidecrypto.xyz")

sql = """
WITH RankedBalances AS (
    SELECT
        USER_ADDRESS,
        block_number,
        balance,
        contract_address,
        ROW_NUMBER() OVER(PARTITION BY USER_ADDRESS, CONTRACT_ADDRESS ORDER BY block_number DESC) as rank
    FROM
        ethereum.core.fact_token_balances
    WHERE
        (CONTRACT_ADDRESS = lower('0x8659fc767cad6005de79af65dafe4249c57927af') OR
        CONTRACT_ADDRESS = lower('0xBCe0Cf87F513102F22232436CCa2ca49e815C3aC') OR
        CONTRACT_ADDRESS = lower('0x389fB29230D02e67eB963C1F5A00f2b16f95BEb7') OR
        CONTRACT_ADDRESS = lower('0x4de2ab8177fc5ffafb7ad11fb8beb1e5fb95f71c') )
        AND
        BLOCK_NUMBER <= 17830755
)

SELECT
    USER_ADDRESS,
    block_number,
    balance,
    contract_address
FROM
    RankedBalances
WHERE
    rank = 1 
"""

res = flipside.query(sql)

In [33]:
TOKEN_MAP = {'0x8659fc767cad6005de79af65dafe4249c57927af': 'ucvx',
 '0xbce0cf87f513102f22232436cca2ca49e815c3ac': 'pxcvx',
 '0x389fb29230d02e67eb963c1f5a00f2b16f95beb7': 'lpxcvx',
 '0x4de2ab8177fc5ffafb7ad11fb8beb1e5fb95f71c': 'curve lp'}

In [62]:
df = pd.DataFrame(data=res.rows, columns=res.columns)
df['balance'] = df['balance'] * 1e-18
df['token'] = df['contract_address'].apply(lambda x: TOKEN_MAP[x])
df = df[df['balance'] > 1] # filter out dust holders
df.reset_index(inplace=True)
df

,index,user_address,block_number,balance,contract_address,__row_index,token
0,2,0x77e510353b9e75b3a8fbed61d0cef4849095773c,15182613,121.372655,0x8659fc767cad6005de79af65dafe4249c57927af,2,ucvx
1,6,0xc55e3bc5b6ee19d8d9a2fd233848cca2842eec59,15208412,186.768065,0x8659fc767cad6005de79af65dafe4249c57927af,6,ucvx
2,8,0x090e1fdc0cb866317751f0621884a203a8d797aa,17769170,4.122318,0x8659fc767cad6005de79af65dafe4249c57927af,8,ucvx
3,9,0x034ea2c4f55ae1c613ce1dc6ae231bac1b1ab10b,17230558,4187.837649,0x8659fc767cad6005de79af65dafe4249c57927af,9,ucvx
4,10,0x8ae6fd8eb06b3a02e3688196cac66de3f4edfe9e,15227620,33.418068,0x8659fc767cad6005de79af65dafe4249c57927af,10,ucvx
...,...,...,...,...,...,...,...
838,2067,0x8a72c47f190f2118227a83da784a2ce45c54f484,15758473,1021.553748,0x8659fc767cad6005de79af65dafe4249c57927af,2067,ucvx
839,2068,0xfab6735cfadd7b3fbb4a1383571629d576801d9c,15176170,629.318804,0x8659fc767cad6005de79af65dafe4249c57927af,2068,ucvx
840,2070,0xfb529f55f4a39d9a336cd1a15b489f07679f4932,15234058,143.67769,0x8659fc767cad6005de79af65dafe4249c57927af,2070,ucvx
841,2071,0x2c3545ceaed6d3cb326486ce01f156282fc7f9a5,15182613,78.873471,0x8659fc767cad6005de79af65dafe4249c57927af,2071,ucvx


In [63]:
df['balance'].quantile([.1, .2, .3, .4, .5, .6, .7, .8, .9])

0.1      14.800963
0.2      42.243721
0.3      67.980345
0.4     112.169609
0.5     156.192959
0.6     263.876846
0.7     489.241576
0.8     1056.54337
0.9    3085.181236
Name: balance, dtype: object

## uCVX

Get the uCVX to CVX exchange rate at the snapshot block

In [64]:
from web3 import Web3

ALCHEMY_API_KEY = os.getenv("WEB3_ALCHEMY_API_KEY")
w3 = Web3(Web3.HTTPProvider(f'https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}'))
ABI = [
    {
        "constant": True,
        "inputs": [{"name": "shares", "type": "uint256"}],
        "name": "convertToAssets",
        "outputs": [{"name": "", "type": "uint256"}],
        "payable": False,
        "stateMutability": "view",
        "type": "function"
    }
]

UCVX_CONTRACT_ADDRESS = "0x8659Fc767cad6005de79AF65dAfE4249C57927AF"
contract = w3.eth.contract(address=UCVX_CONTRACT_ADDRESS, abi=ABI)
rate = contract.functions.convertToAssets(int(1e18)).call(block_identifier=17830755)

In [65]:
UCVX_MERKLE = '0x6788234f40931ca615b0d221c1afbf0ec07afcc5'
ucvx_df = df[(df['user_address'] != UCVX_MERKLE) & (df['token'] == 'ucvx')].copy()
ucvx_df['balance'] = ucvx_df['balance'] * (rate * 1e-18)
ucvx_df

,index,user_address,block_number,balance,contract_address,__row_index,token
0,2,0x77e510353b9e75b3a8fbed61d0cef4849095773c,15182613,162.914198,0x8659fc767cad6005de79af65dafe4249c57927af,2,ucvx
1,6,0xc55e3bc5b6ee19d8d9a2fd233848cca2842eec59,15208412,250.692129,0x8659fc767cad6005de79af65dafe4249c57927af,6,ucvx
2,8,0x090e1fdc0cb866317751f0621884a203a8d797aa,17769170,5.533241,0x8659fc767cad6005de79af65dafe4249c57927af,8,ucvx
3,9,0x034ea2c4f55ae1c613ce1dc6ae231bac1b1ab10b,17230558,5621.185503,0x8659fc767cad6005de79af65dafe4249c57927af,9,ucvx
4,10,0x8ae6fd8eb06b3a02e3688196cac66de3f4edfe9e,15227620,44.855884,0x8659fc767cad6005de79af65dafe4249c57927af,10,ucvx
...,...,...,...,...,...,...,...
838,2067,0x8a72c47f190f2118227a83da784a2ce45c54f484,15758473,1371.195256,0x8659fc767cad6005de79af65dafe4249c57927af,2067,ucvx
839,2068,0xfab6735cfadd7b3fbb4a1383571629d576801d9c,15176170,844.712244,0x8659fc767cad6005de79af65dafe4249c57927af,2068,ucvx
840,2070,0xfb529f55f4a39d9a336cd1a15b489f07679f4932,15234058,192.853452,0x8659fc767cad6005de79af65dafe4249c57927af,2070,ucvx
841,2071,0x2c3545ceaed6d3cb326486ce01f156282fc7f9a5,15182613,105.869054,0x8659fc767cad6005de79af65dafe4249c57927af,2071,ucvx


In [66]:
ucvx_df[['user_address', 'balance']].to_json(orient='records')

'[{"user_address":"0x77e510353b9e75b3a8fbed61d0cef4849095773c","balance":162.9141976823},{"user_address":"0xc55e3bc5b6ee19d8d9a2fd233848cca2842eec59","balance":250.6921291445},{"user_address":"0x090e1fdc0cb866317751f0621884a203a8d797aa","balance":5.5332411834},{"user_address":"0x034ea2c4f55ae1c613ce1dc6ae231bac1b1ab10b","balance":5621.1855025022},{"user_address":"0x8ae6fd8eb06b3a02e3688196cac66de3f4edfe9e","balance":44.855883555},{"user_address":"0x8b7bdb4db9072902873556d60534a05a1a551c65","balance":88.4838019022},{"user_address":"0xf23e1befc889ac30991762ed44fbd8ebf260419b","balance":3071.5755010817},{"user_address":"0x304016db3eea6acccd988037fc9734a18d9dbea1","balance":105.0247329852},{"user_address":"0xe4e52c6a0e560397412e0cf995044ac2377a480d","balance":79.1553284698},{"user_address":"0xb47e9e28543e7810d5752a6b8bb53c5372300758","balance":879.3772416927},{"user_address":"0xdf5d7312c229dd782bc1311cd98b57749e343c48","balance":843.561150206},{"user_address":"0x4ff60a0e7f3c35a00537e5fa824

## uCVX distributor allocations

In [81]:
import json
with open('merkle_ucvx.json', 'r') as file:
    data = json.load(file)

In [82]:
dis_df = pd.DataFrame(data=data, columns=['user_address', 'balance'])

In [85]:
dis_df['user_address'] = dis_df['user_address'].str.lower()

## LPs

In [67]:
df[df['token'] == 'lpxcvx']

,index,user_address,block_number,balance,contract_address,__row_index,token
409,997,0xa52fd396891e7a74b641a2cb1a6999fcf56b077e,17587962,34863.915754,0x389fb29230d02e67eb963c1f5a00f2b16f95beb7,997,lpxcvx
822,2032,0x72725c0c879489986d213a9a6d2116de45624c1c,17830116,129229.825129,0x389fb29230d02e67eb963c1f5a00f2b16f95beb7,2032,lpxcvx


In [68]:
df[df['token'] == 'curve lp']

,index,user_address,block_number,balance,contract_address,__row_index,token
358,860,0xecb456ea5365865ebab8a2661b0c503410e9b347,17830116,325.198472,0x4de2ab8177fc5ffafb7ad11fb8beb1e5fb95f71c,860,curve lp
396,957,0xe92cd2b13b6a849316b86d36cac887a629f1e00e,16891478,10.0,0x4de2ab8177fc5ffafb7ad11fb8beb1e5fb95f71c,957,curve lp
749,1874,0xa52fd396891e7a74b641a2cb1a6999fcf56b077e,17438262,84996.255653,0x4de2ab8177fc5ffafb7ad11fb8beb1e5fb95f71c,1874,curve lp


We discount the dust & curve fee collection address

In [70]:
84996.255653 / sum(df[df['token'] == 'curve lp']['balance']) * 129229.825129

128722.18535680581

In [72]:
lp = 34863.915754 + 128722.18535680581
lp

163586.10111080582

In [93]:
lp_df = pd.DataFrame(data=[['0xa52fd396891e7a74b641a2cb1a6999fcf56b077e', lp]], columns=['user_address', 'balance'])
lp_df

,user_address,balance
0,0xa52fd396891e7a74b641a2cb1a6999fcf56b077e,163586.101111


## pxCVX

In [76]:
df[df['token'] == 'pxcvx'][['user_address', 'balance']].to_json(orient='records')

'[{"user_address":"0xa52fd396891e7a74b641a2cb1a6999fcf56b077e","balance":14881.0821781168},{"user_address":"0x2e1d4bb1c6d285368aa8d875b1d2795a70b55452","balance":10.8751},{"user_address":"0xe344489b0812e76657e59f4d14b2d3c4fac2a8a3","balance":374.73},{"user_address":"0x94e4aa6eef65b48fea01d5a5cce0bc0dffb66f43","balance":30.137489074},{"user_address":"0x2b4cce029aaf701395a70e6888e2c59509201847","balance":143.2123191853},{"user_address":"0xa8a29c94820949c4d3348bcf2b2753096ca85739","balance":378.4108557166},{"user_address":"0xe5504e8109e032bc5d961e4f3c358705104e198e","balance":55.1480231171},{"user_address":"0x8df24569a74e094bf75e17d1f0c238d39b5e17b5","balance":3.0131},{"user_address":"0xe47fd677ba6d3738cec98aa18bb6fa015b6f7bda","balance":8.1287179411},{"user_address":"0xa694e6603e1f0cd880e3fa6e047da12ddd90395d","balance":10.0},{"user_address":"0x434057ad8ff687846789427b7a018328613c6686","balance":158.9969741272},{"user_address":"0xc52815221d716665ab84470f40e57037e473d14f","balance":241.97

In [78]:
df[df['token'] == 'pxcvx']['balance'].quantile([.1, .2, .3, .4, .5, .6, .7, .8, .9])

0.1        4.658915
0.2       10.633283
0.3       28.051128
0.4       50.881629
0.5      148.697308
0.6      254.586048
0.7      490.109494
0.8     1450.979288
0.9    10945.196643
Name: balance, dtype: object

## Aggregate

In [101]:
agg =  pd.concat([dis_df, # ucvx distributor
         df[df['token'] == 'pxcvx'][['user_address', 'balance']], # pxcvx
         lp_df, # liquidity providers
         ucvx_df[['user_address', 'balance']]], #ucvx
          axis=0
         )
agg = agg.groupby('user_address').sum().reset_index()
agg

,user_address,balance
0,0x0056d1fd2ca3c0f3a7b6ed6cdd1f1f104b4bf9a9,10.267250
1,0x0058ed26f567453ca826b60dbf09ef5465b7feab,16.856335
2,0x006217c11f51611d9848d17550bc4b03fe02deb0,194.466152
3,0x006b4b47c7f404335c87e85355e217305f97e789,657.877195
4,0x0079f92ba09c15b21a62ba97caaf2e3aa4531643,56.550508
...,...,...
1403,0xff18311ccdbe86e54d266f07eaec2668bee9f9f3,7.221864
1404,0xff597632d59c49e918fe26cda385f37a0db8a713,292.955568
1405,0xff6a59e3b0955bf1774e10ff8fc059938cddbca0,87.216106
1406,0xffcf75bd0b598fb0ae994d81955da7c032e7d1c1,1.933797


In [115]:
final = agg.set_index('user_address')['balance'].to_dict()
with open('allocations.json', 'w') as fp:
    json.dump(final, fp, indent=4)